In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, optimizers, callbacks, backend, regularizers

# --- CONFIGURATION ---
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30
SEED = 42
NUM_CLASSES = 5

# --- (CLAHE) ---
# CLAHE to normalize the lighting differences 
# between the training set and the 'Alien' test set.
def preprocess_image_robust(img):
    # 1. Ensure 8-bit for OpenCV
    img = img.astype('uint8')
    
    # 2. Convert to LAB color space (L = Lightness)
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    
    # 3. Apply CLAHE to Lightness channel only
    # This fixes contrast without messing up colors
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    
    # 4. Merge back and convert to RGB
    merged = cv2.merge((l, a, b))
    final = cv2.cvtColor(merged, cv2.COLOR_LAB2RGB)
    
    # 5. Normalize to 0-1
    return final.astype('float32') / 255.0

print("Environment Ready & CLAHE Preprocessor Defined.")

Environment Ready & CLAHE Preprocessor Defined.


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Prepare Data Paths
dataset_path = './dataset'
classes = ['cloudy', 'foggy', 'rainy', 'shine', 'sunrise']
filepaths = []
labels = []

for idx, cls in enumerate(classes):
    cls_folder = os.path.join(dataset_path, cls)
    for img_name in os.listdir(cls_folder):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            filepaths.append(os.path.join(cls_folder, img_name))
            labels.append(str(idx))

# Create DataFrame
full_train_df = pd.DataFrame({'filepath': filepaths, 'label': labels})
full_train_df = full_train_df.sample(frac=1, random_state=SEED).reset_index(drop=True)

print(f"Total Training Images: {len(full_train_df)}")

# 2. Robust Augmentation
#  geometric augmentations to teach the model 
# invariance, but kept color augmentation mild to preserve weather features.
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image_robust, # Apply CLAHE
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = train_datagen.flow_from_dataframe(
    full_train_df,
    x_col='filepath',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

print("Data Generator Ready.")

Total Training Images: 1500
Found 1500 validated image filenames belonging to 5 classes.
Data Generator Ready.


In [3]:
def squeeze_excite_block(input_tensor, ratio=16):
    # the network to recalibrate features,
    # effectively 'paying attention' to important weather cues
    filters = input_tensor.shape[-1]
    
    se = layers.GlobalAveragePooling2D()(input_tensor)
    se = layers.Reshape((1, 1, filters))(se)
    
    # Squeeze
    se = layers.Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    # Excite
    se = layers.Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    
    # Scale
    x = layers.Multiply()([input_tensor, se])
    return x

def resnet_se_block(x, filters, stride=1):
    shortcut = x
    
    # First Conv
    x = layers.Conv2D(filters, (3, 3), strides=stride, padding='same', use_bias=False, kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # Second Conv
    x = layers.Conv2D(filters, (3, 3), strides=1, padding='same', use_bias=False, kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    
    #  SE Block 
    x = squeeze_excite_block(x)
    
    # Shortcut handling
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), strides=stride, padding='same', use_bias=False, kernel_initializer='he_normal')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def build_se_resnet():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    
    # Stem
    x = layers.Conv2D(64, (7, 7), strides=2, padding='same', use_bias=False, kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)
    
    # Body (3 Stages)
    x = resnet_se_block(x, 64)
    x = resnet_se_block(x, 64)
    
    x = resnet_se_block(x, 128, stride=2)
    x = resnet_se_block(x, 128)
    
    x = resnet_se_block(x, 256, stride=2)
    x = resnet_se_block(x, 256)
    
    # Head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x) # Regularization
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = models.Model(inputs, outputs, name="Custom_SE_ResNet")
    return model

# Build and Compile
backend.clear_session()
model = build_se_resnet()

# Label Smoothing: Helps with noisy data (Cloudy vs Foggy confusion)
optimizer = optimizers.Adam(learning_rate=1e-3)
model.compile(
    optimizer=optimizer, 
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1), 
    metrics=['accuracy']
)

print("Custom SE-ResNet Built.")
model.summary()


Custom SE-ResNet Built.


Model: "Custom_SE_ResNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      9,408 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 56, 56,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 56, 56,    │     36,864 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │     36,864 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 64)  │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1, 1, 4)   │        256 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1, 1, 64)  │        256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 56, 56,    │          0 │ multiply[0][0],   │
│                     │ 64)               │            │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 56, 56,    │          0 │ add[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 56, 56,    │     36,864 │ activation_2[0][

 Total params: 2,810,053 (10.72 MB)

 Trainable params: 2,805,573 (10.70 MB)

 Non-trainable params: 4,480 (17.50 KB)

In [4]:
# Callbacks
checkpoint = callbacks.ModelCheckpoint(
    'best_se_model.h5', 
    monitor='loss', # Monitoring loss because we have no validation split
    save_best_only=True, 
    mode='min',
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='loss', 
    factor=0.2, 
    patience=3, 
    min_lr=1e-6, 
    verbose=1
)

print(f"Starting Training for {EPOCHS} Epochs on 100% Data...")

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    callbacks=[checkpoint, reduce_lr],
    verbose=1
)

Starting Training for 30 Epochs on 100% Data...
Epoch 1/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6090 - loss: 1.2881
Epoch 1: loss improved from None to 1.07737, saving model to best_se_model.h5



Epoch 1: finished saving model to best_se_model.h5
47/47 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.7027 - loss: 1.0774 - learning_rate: 0.0010
Epoch 2/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8130 - loss: 0.8340
Epoch 2: loss improved from 1.07737 to 0.83936, saving model to best_se_model.h5



Epoch 2: finished saving model to best_se_model.h5
47/47 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.8093 - loss: 0.8394 - learning_rate: 0.0010
Epoch 3/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8332 - loss: 0.7574
Epoch 3: loss improved from 0.83936 to 0.73867, saving model to best_se_model.h5



Epoch 3: finished saving model to best_se_model.h5
47/47 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - accuracy: 0.8560 - loss: 0.7387 - learning_rate: 0.0010
Epoch 4/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8663 - loss: 0.7051
Epoch 4: loss improved from 0.73867 to 0.68315, saving model to best_se_model.h5



Epoch 4: finished saving model to best_se_model.h5
47/47 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.8813 - loss: 0.6832 - learning_rate: 0.0010
Epoch 5/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8829 - loss: 0.6819
Epoch 5: loss improved from 0.68315 to 0.67170, saving model to best_se_model.h5


...


Epoch 25: finished saving model to best_se_model.h5
47/47 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.9687 - loss: 0.4828 - learning_rate: 2.0000e-04
Epoch 26/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9677 - loss: 0.4782
Epoch 26: loss did not improve from 0.48280
47/47 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.9660 - loss: 0.4852 - learning_rate: 2.0000e-04
Epoch 27/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9771 - loss: 0.4704
Epoch 27: loss improved from 0.48280 to 0.47770, saving model to best_se_model.h5



Epoch 27: finished saving model to best_se_model.h5
47/47 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.9713 - loss: 0.4777 - learning_rate: 2.0000e-04
Epoch 28/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9763 - loss: 0.4731
Epoch 28: loss improved from 0.47770 to 0.47489, saving model to best_se_model.h5



Epoch 28: finished saving model to best_se_model.h5
47/47 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.9727 - loss: 0.4749 - learning_rate: 2.0000e-04
Epoch 29/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9702 - loss: 0.4694
Epoch 29: loss improved from 0.47489 to 0.47112, saving model to best_se_model.h5



Epoch 29: finished saving model to best_se_model.h5
47/47 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.9707 - loss: 0.4711 - learning_rate: 2.0000e-04
Epoch 30/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9828 - loss: 0.4595
Epoch 30: loss improved from 0.47112 to 0.46784, saving model to best_se_model.h5



Epoch 30: finished saving model to best_se_model.h5
47/47 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.9793 - loss: 0.4678 - learning_rate: 2.0000e-04


In [5]:
print("--- STARTING FINAL INFERENCE ---")

# 1. Load Best Model
final_model = models.load_model('best_se_model.h5')

# 2. Load Test Data
test_csv_path = './dataset/test.csv'
alien_path = './dataset/alien_test'
test_df = pd.read_csv(test_csv_path)

ids = []
labels = []

# 3. Inference Loop
for index, row in test_df.iterrows():
    img_filename = row['Image_id']
    full_path = os.path.join(alien_path, img_filename)
    
    try:
        # A. Load Image
        img_raw = cv2.imread(full_path)
        if img_raw is None:
            raise ValueError("Image not found")
            
        img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
        
        # B. Resize (CRITICAL FIX)
        img_resized = cv2.resize(img_rgb, (IMG_SIZE, IMG_SIZE))
        
        # C. Preprocess (CLAHE)
        # We must use the exact same function we used for training
        img_pre = preprocess_image_robust(img_resized)
        
        # D. Batch Dimension
        img_batch = np.expand_dims(img_pre, axis=0)
        
        # E. TTA (Flip)
        img_flipped = np.flip(img_batch, axis=2) # Horizontal Flip
        
        # F. Predict (Average of Normal + Flipped)
        pred_normal = final_model.predict(img_batch, verbose=0)
        pred_flipped = final_model.predict(img_flipped, verbose=0)
        
        avg_pred = (pred_normal + pred_flipped) / 2.0
        final_class = np.argmax(avg_pred)
        
        ids.append(row['id'])
        labels.append(final_class)
        
    except Exception as e:
        print(f"Error on {img_filename}: {e}")
        ids.append(row['id'])
        labels.append(0) # Fallback

    if index % 10 == 0:
        print(f"Processed {index}/{len(test_df)}")

# 4. Save
submission = pd.DataFrame({'id': ids, 'labels': labels})
submission['id'] = submission['id'].astype(int)
submission['labels'] = submission['labels'].astype(int)

submission.to_csv('submission.csv', index=False)

print("\nSUCCESS! submission.csv created.")
print(submission['labels'].value_counts())

--- STARTING FINAL INFERENCE ---


Processed 0/30
Processed 10/30
Processed 20/30

SUCCESS! submission_final_se_resnet.csv created.
labels
1    9
2    8
4    7
0    3
3    3
Name: count, dtype: int64
